Imports

In [231]:
import json
import numpy as np
from gensim import corpora, models, similarities
from nltk.tokenize import word_tokenize
import nltk
import os
import shutil
import logging

Carregar variáveis globais

In [232]:
%store -r id_consulta

Configuração do Logger

In [233]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

Carregando .jsons

In [234]:
with open('map_documentos.json', 'r', encoding='utf-8') as f:
    map_documentos = json.load(f)
with open('map_consultas.json', 'r', encoding='utf-8') as f:
    map_consultas = json.load(f)

Download de dependências do NLTK
   

In [235]:
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')

Indexar o map_documentos com Vetorial (TF-IDF)

In [236]:
# Crie uma lista com os textos dos documentos. A ordem será preservada.
lista_de_documentos = list(map_documentos.values())
# Crie o corpus tokenizado a partir dessa lista
corpus_tokenizado = [doc.lower().split(" ") for doc in lista_de_documentos]
# Guardar os IDs dos documentos na mesma ordem para mapeamento posterior
ids_documentos = list(map_documentos.keys())

Construção do modelo Vetorial

In [237]:
logging.info("Indexando o corpus com o modelo Vetorial (TF-IDF)...")
dictionary = corpora.Dictionary(corpus_tokenizado)
bow_corpus = [dictionary.doc2bow(text) for text in corpus_tokenizado]
tfidf_model = models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf_model[bow_corpus]

# Usando 'similarities.Similarity' para eficiência de memória
temp_folder = './temp_vetorial_index'
if os.path.exists(temp_folder):
    shutil.rmtree(temp_folder)
os.makedirs(temp_folder)
output_prefix = os.path.join(temp_folder, 'shard')
index = similarities.Similarity(output_prefix, tfidf_corpus, num_features=len(dictionary))


2025-07-22 11:15:17 - INFO - Indexando o corpus com o modelo Vetorial (TF-IDF)...
2025-07-22 11:15:17 - INFO - adding document #0 to Dictionary<0 unique tokens: []>
2025-07-22 11:15:17 - INFO - adding document #10000 to Dictionary<8953 unique tokens: ['ajepidem', 'environmental', 'epidemiology', 'john', 'snow']...>
2025-07-22 11:15:17 - INFO - adding document #20000 to Dictionary<12182 unique tokens: ['ajepidem', 'environmental', 'epidemiology', 'john', 'snow']...>
2025-07-22 11:15:17 - INFO - adding document #30000 to Dictionary<16741 unique tokens: ['ajepidem', 'environmental', 'epidemiology', 'john', 'snow']...>
2025-07-22 11:15:17 - INFO - adding document #40000 to Dictionary<19925 unique tokens: ['ajepidem', 'environmental', 'epidemiology', 'john', 'snow']...>
2025-07-22 11:15:17 - INFO - adding document #50000 to Dictionary<22221 unique tokens: ['ajepidem', 'environmental', 'epidemiology', 'john', 'snow']...>
2025-07-22 11:15:17 - INFO - adding document #60000 to Dictionary<25505

Função que retorna Top N documentos de uma consulta Vetorial

In [238]:
def obter_top_n_ids_vetorial(consulta_tokenizada, n):
    # 1. Converte a consulta para o espaço vetorial TF-IDF
    query_bow = dictionary.doc2bow(consulta_tokenizada)
    query_tfidf = tfidf_model[query_bow]

    # 2. Calcula os scores (similaridades) para cada documento no corpus
    doc_scores = index[query_tfidf]

    # 3. Pega os índices dos scores em ordem decrescente de pontuação
    top_n_indices = np.argsort(doc_scores)[::-1]

    # 4. Retorna apenas os 'n' primeiros IDs da lista ordenada
    # Pega o ID original da lista 'ids_documentos' e converte para inteiro
    return [int(ids_documentos[idx]) for idx in top_n_indices[:n]]

Executar consulta e gerar os resultados top 50

In [ ]:
# Escolha uma consulta para testar
consulta_texto = map_consultas[id_consulta]
consulta_tokenizada = consulta_texto.lower().split(" ")
# Chame a função, que retornará os IDs dos documentos mais relevantes.
resultados_vetorial = obter_top_n_ids_vetorial(consulta_tokenizada, n=100)

2025-07-22 11:15:33 - INFO - creating sparse index
2025-07-22 11:15:33 - INFO - creating sparse matrix from corpus


2025-07-22 11:15:33 - INFO - PROGRESS: at document #0/31187
2025-07-22 11:15:33 - INFO - PROGRESS: at document #10000/31187
2025-07-22 11:15:33 - INFO - PROGRESS: at document #20000/31187
2025-07-22 11:15:33 - INFO - PROGRESS: at document #30000/31187
2025-07-22 11:15:33 - INFO - created <31187x44923 sparse matrix of type '<class 'numpy.float32'>'
	with 256208 stored elements in Compressed Sparse Row format>
2025-07-22 11:15:33 - INFO - creating sparse shard #4
2025-07-22 11:15:33 - INFO - saving index shard to ./temp_vetorial_index\shard.4
2025-07-22 11:15:33 - INFO - SparseMatrixSimilarity lifecycle event {'fname_or_handle': './temp_vetorial_index\\shard.4', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2025-07-22T11:15:33.648308', 'gensim': '4.3.3', 'python': '3.12.8 | Intel Corporation | (main, Apr  2 2025, 09:07:08) [MSC v.1940 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.26100-SP0', 'event': 'saving'}
2025-07-22 11:15:33 - INFO - saved ./temp_

Salvando resultados_vetorial como .json

In [240]:
nome_arquivo_saida = 'resultados_tfidf.json'
logging.info(f"Salvando resultados em '{nome_arquivo_saida}'...")
try:
    with open(nome_arquivo_saida, 'w', encoding='utf-8') as out_file_opened:
        json.dump(
            resultados_vetorial,    # Dados a serem salvos
            out_file_opened,        # Objeto do arquivo
            indent=2,               # Indentação para melhorar legibilidade
            ensure_ascii=False      # Permite caracteres não-ASCII
        )
    logging.info("Arquivo de resultados salvo com sucesso!")
except Exception as e:
    logging.error(f"Erro ao salvar o arquivo JSON: {e}")
finally:
    # Limpa a pasta temporária do índice
    if os.path.exists(temp_folder):
        shutil.rmtree(temp_folder)

2025-07-22 11:15:33 - INFO - Salvando resultados em 'resultados_tfidf.json'...
2025-07-22 11:15:33 - INFO - Arquivo de resultados salvo com sucesso!
